Importar Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

Importar base de dados original

In [ ]:
path = "databases/MICRODADOS_ENEM_2023.csv" # 1.735 GB (Excluído)
try:
    df = pd.read_csv(path, sep=";", encoding="latin-1", low_memory=False)
    display(df) # 3933955 rows × 76 columns (3m 19.3s)
except:
    print(f"Arquivo {path} não encontrado.")
    df = pd.read_csv("databases/ENEM_2023_SP_1.csv")

Filtrando estudantes da cidade de São Paulo, que não são treineros e não faltaram ou foram eliminados em nenhuma prova.

In [ ]:
def filter_sp_students(df:pd.DataFrame):
    filter = (
        (df["NO_MUNICIPIO_ESC"] == "São Paulo") & # Estudou na cidade de São Paulo 
        (df["IN_TREINEIRO"] == 0) &               # Não é treineiro
        (df["TP_PRESENCA_CH"] == 1) &             # Não faltou, nem foi eliminado da prova
        (df["TP_PRESENCA_CN"] == 1) &             # Não faltou, nem foi eliminado da prova
        (df["TP_PRESENCA_LC"] == 1) &             # Não faltou, nem foi eliminado da prova
        (df["TP_PRESENCA_MT"] == 1)               # Não faltou, nem foi eliminado da prova
    )
    return df[filter]

df = filter_sp_students(df) 
df.to_csv("databases/ENEM_2023_SP_1.csv", index=False)

display(df) # 35416 rows × 76 columns

Excluindo colunas "desnecessárias", mas mantendo aquelas que serão utilizadas para calcular outros campos. Também feita a renomeação de algumas colunas.

In [ ]:
def filter_columns(df:pd.DataFrame):
    cols = ["TP_FAIXA_ETARIA", "TP_SEXO", "TP_COR_RACA", "TP_ENSINO", "TP_DEPENDENCIA_ADM_ESC", "NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_REDACAO", "Q001", "Q002", "Q005", "Q006", "Q010", "Q011", "Q022", "Q024", "Q025"]
    df = df[cols]

    rename_dict = {
        "Q001": "ESCOLARIDADE_PAI",
        "Q002": "ESCOLARIDADE_MAE",
        "Q005": "N_PESSOAS_CASA",
        "Q006": "RENDA_MENSAL_CASA",
        "Q010": "N_CARRO_CASA",
        "Q011": "N_MOTO_CASA",
        "Q022": "N_CELULAR_CASA",
        "Q024": "N_COMP_CASA",
        "Q025": "INTERNET_CASA"
    }

    return df.rename(columns=rename_dict)

df = pd.read_csv("databases/ENEM_2023_SP_1.csv")
df = filter_columns(df)
df.to_csv("databases/ENEM_2023_SP_2.csv", index=False)

display(df) # 35416 rows × 20 columns

Sequência de value counts para observar proporções de certas categorias na amostra.

In [ ]:
def all_value_counts(df:pd.DataFrame):
    categoric_cols = ["TP_FAIXA_ETARIA", "TP_SEXO", "TP_COR_RACA", "TP_ENSINO", "TP_DEPENDENCIA_ADM_ESC", "ESCOLARIDADE_PAI", "ESCOLARIDADE_MAE", "N_PESSOAS_CASA", "RENDA_MENSAL_CASA", "N_CARRO_CASA", "N_MOTO_CASA", "N_CELULAR_CASA", "N_COMP_CASA", "INTERNET_CASA"]
    for col in categoric_cols:
        print(df[col].value_counts(dropna=False))
        print()

df = pd.read_csv("databases/ENEM_2023_SP_2.csv")
all_value_counts(df)

In [ ]:
def drop_not_available_data(df:pd.DataFrame):
    filter = (
        (df["TP_COR_RACA"] == 0) |
        (df["TP_COR_RACA"] == 6) |
        (df["TP_ENSINO"] != 1.0) |
        (df["ESCOLARIDADE_MAE"] == "H") | 
        (df["ESCOLARIDADE_PAI"] == "H")
    )

    df = df.drop(columns=["TP_ENSINO"])
    return df.loc[~filter].reset_index(drop=True)

df = pd.read_csv("databases/ENEM_2023_SP_2.csv")
df = drop_not_available_data(df)
df.to_csv("databases/ENEM_2023_SP_3.csv", index=False)

display(df) # 31731 rows × 19 columns (3703 linhas escluídas)

Obtendo colunas de estimação para renda e faixa etária, além de outra para indicar se a família possui ou não veículo próprio.

In [ ]:
def create_estimative_and_simplified_columns(df:pd.DataFrame):
    map_renda = {
        "A": 0,                       # Nenhuma renda
        "B": (0 + 1320)/2,            # Até 1320 (médio = 660)
        "C": (1320.01 + 1980)/2,      # 1650
        "D": (1980.01 + 2640)/2,      # 2310
        "E": (2640.01 + 3300)/2,      # 2970
        "F": (3300.01 + 3960)/2,      # 3630
        "G": (3960.01 + 5280)/2,      # 4620
        "H": (5280.01 + 6600)/2,      # 5940
        "I": (6600.01 + 7920)/2,      # 7260
        "J": (7920.01 + 9240)/2,      # 8580
        "K": (9240.01 + 10560)/2,     # 9900
        "L": (10560.01 + 11880)/2,    # 11220
        "M": (11880.01 + 13200)/2,    # 12540
        "N": (13200.01 + 15840)/2,    # 14520
        "O": (15840.01 + 19800)/2,    # 17820
        "P": (19800.01 + 26400)/2,    # 23100
        "Q": (26400.01 + 33000)/2     # Arbitrário, assume R$ 33000 pois é a mesma distância do intervalo anterior
    }
    map_idade = {
        1: 16,               # Menor de 17 anos -> assume 16
        2: 17,
        3: 18,
        4: 19,
        5: 20,
        6: 21,
        7: 22,
        8: 23,
        9: 24,
        10: 25,
        11: (26 + 30)/2,   # 28
        12: (31 + 35)/2,   # 33
        13: (36 + 40)/2,   # 38
        14: (41 + 45)/2,   # 43
        15: (46 + 50)/2,   # 48
        16: (51 + 55)/2,   # 53
        17: (56 + 60)/2,   # 58
        18: (61 + 65)/2,   # 63
        19: (66 + 70)/2,   # 68
        20: (71 + 75)/2    # Maior de 70 anos
    }

    df["EST_RENDA_PER_CAP"] = (df["RENDA_MENSAL_CASA"].map(map_renda))/df["N_PESSOAS_CASA"]
    df["EST_IDADE"] = df["TP_FAIXA_ETARIA"].map(map_idade)
    df["VEICULO_PROPRIO_FAM"] = (df["N_CARRO_CASA"] != "A") | (df["N_CARRO_CASA"] != "A")

    return df.drop(columns=["RENDA_MENSAL_CASA", "N_PESSOAS_CASA", "TP_FAIXA_ETARIA", "N_CARRO_CASA", "N_MOTO_CASA"])

df = pd.read_csv("databases/ENEM_2023_SP_3.csv")
df = create_estimative_and_simplified_columns(df)
df.to_csv("databases/ENEM_2023_SP_4.csv", index=False)

display(df) # 31731 rows × 16 columns